Notebook created: 2018-02-15 22:32:57  
Generated from: _build_py/py/mle.rst  

In [ ]:
from numpy import exp
from scipy.misc import factorial
import matplotlib.pyplot as plt

poisson_pmf = lambda y, mu: mu**y / factorial(y) * exp(-mu)
y_values = range(0, 25)

fig, ax = plt.subplots(figsize=(12, 8))

for mu in [1, 5, 10]:
    distribution = []
    for y_i in y_values:
        distribution.append(poisson_pmf(y_i, mu))
    ax.plot(y_values,
            distribution,
            label=('$\mu$=' + str(mu)),
            alpha=0.5,
            marker='o',
            markersize=8)

ax.grid()
ax.set_xlabel('$y$', fontsize=14)
ax.set_ylabel('$f(y \mid \mu)$', fontsize=14)
ax.axis(xmin=0, ymin=0)
ax.legend(fontsize=14)

plt.show()

In [ ]:
import pandas as pd
pd.options.display.max_columns = 10

# Load in data and view
df = pd.read_stata('https://github.com/QuantEcon/QuantEcon.lectures.code/raw/master/mle/fp.dta')
df.head()

In [ ]:
numbil0_2008 = df[(df['year'] == 2008) & (
    df['country'] != 'United States')].loc[:, 'numbil0']

plt.subplots(figsize=(12, 8))
plt.hist(numbil0_2008, bins=30)
plt.xlim(xmin=0)
plt.grid()
plt.xlabel('Number of billionaires in 2008')
plt.ylabel('Count')
plt.show()

In [ ]:
import numpy as np

y_values = range(0, 20)

# Define a parameter vector with estimates
beta = np.array([0.26, 0.18, 0.25, -0.1, -0.22]).T

# Create some observations X
datasets = [np.array([0, 1, 1, 1, 2]),
            np.array([2, 3, 2, 4, 0]),
            np.array([3, 4, 5, 3, 2]),
            np.array([6, 5, 4, 4, 7])]


fig, ax = plt.subplots(figsize=(12, 8))

for X in datasets:
    mu = exp(X @ beta)
    distribution = []
    for y_i in y_values:
        distribution.append(poisson_pmf(y_i, mu))
    ax.plot(y_values,
            distribution,
            label=('$\mu_i$=' + str(round(mu, 1))),
            marker='o',
            markersize=8,
            alpha=0.5)

ax.grid()
ax.legend()
ax.set_xlabel('$y \mid x_i$')
ax.set_ylabel('$f(y \mid x_i; \\beta )$')
ax.axis(xmin=0, ymin=0)
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def plot_joint_poisson(mu=7, y_n=20):
    yi_values = np.arange(0, y_n, 1)

    # Create coordinate points of X and Y
    X, Y = np.meshgrid(yi_values, yi_values)

    # Multiply distributions together
    Z = poisson_pmf(X, mu) * poisson_pmf(Y, mu)

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z.T, cmap='terrain', alpha=0.6)
    ax.scatter(X, Y, Z.T, color='black', alpha=0.5, linewidths=1)
    ax.set(xlabel='$y_1$', ylabel='$y_2$')
    ax.set_zlabel('$f(y_1, y_2)$', labelpad=10)
    plt.show()

plot_joint_poisson(mu=7, y_n=20)

In [ ]:
beta = np.linspace(1, 20)
logL = -(beta - 10) ** 2 - 10
dlogL = -2 * beta + 20

fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(12, 8))

ax1.plot(beta, logL, lw=2)
ax2.plot(beta, dlogL, lw=2)

ax1.set_ylabel('$log \mathcal{L(\\beta)}$',
               rotation=0,
               labelpad=35,
               fontsize=15)
ax2.set_ylabel(r'$\frac{dlog \mathcal{L(\beta)}}{d \beta}$ ',
               rotation=0,
               labelpad=35,
               fontsize=19)
ax2.set_xlabel('$\\beta$', fontsize=15)
ax1.grid(), ax2.grid()
plt.axhline(c='black')
plt.show()

In [ ]:
class PoissonRegression:

    def __init__(self, y, X, beta):
        self.X, self.y, self.beta = X, y, beta
        self.n, self.k = X.shape

    def mu(self):
        return np.exp(np.dot(self.X, self.beta.T))

    def logL(self):
        y = self.y
        mu = self.mu()
        return np.sum(y*np.log(mu) - mu - np.log(factorial(y)))

    def G(self):
        mu = self.mu()
        return np.dot(self.y - mu, self.X).reshape(self.k, 1)

    def H(self):
        X = self.X
        mu = self.mu()
        return -np.dot(mu * X.T, X)

In [ ]:
def newton_raphson(model, tol=1e-3, max_iter=1000, display=True):

    i = 0
    error = 100  # Initial error value
    format_string = "{i:<13}{l:<16}{t:<30}"

    # Print header of output
    if display:
        header = format_string.format(i="Iteration_k", l="Log-likelihood", t="Theta")
        print(header)
        print("-" * len(header))

    # While loop runs while any value in error is greater
    # than the tolerance until max iterations are reached
    while np.any(error > tol) and i < max_iter:
        H, G = model.H(), model.G()
        beta_new = model.beta - (np.linalg.inv(H) @ G).T
        error = beta_new - model.beta
        model.beta = beta_new.flatten()

        # Print iterations
        if display:
            beta_list = ['%.4f' % t for t in list(np.round(model.beta, 3))]
            update = format_string.format(i=i, l=round(model.logL(), 8), t=str(beta_list))
            print(update)

        i += 1

    print('Number of iterations: ' + str(i))
    print('beta_hat = ' + str(model.beta))

    return model.beta

In [ ]:
X = np.array([[1, 2, 5],
              [1, 1, 3],
              [1, 4, 2],
              [1, 5, 2],
              [1, 3, 1]])

y = np.array([1, 0, 1, 1, 0])

# Take a guess at initial betas
init_beta = np.array([0.1, 0.1, 0.1])

# Create an object with Poisson model values
poi = PoissonRegression(y, X, beta=init_beta)

# Use newton_raphson to find the MLE
beta_hat = newton_raphson(poi, display=True)

```none
Iteration_k  Log-likelihood  Theta
-----------------------------------------------------------
0            -4.34476224     ['-1.4890', '0.2650', '0.2440']
1            -3.5742413      ['-3.3840', '0.5280', '0.4740']
2            -3.39995256     ['-5.0640', '0.7820', '0.7020']
3            -3.37886465     ['-5.9150', '0.9090', '0.8200']
4            -3.3783559      ['-6.0740', '0.9330', '0.8430']
5            -3.37835551     ['-6.0780', '0.9330', '0.8430']
Number of iterations: 6
beta_hat = [-6.07848205  0.93340226  0.84329625]
```


In [ ]:
poi.G()

```none
array([[ -3.95169226e-07],
       [ -1.00114804e-06],
       [ -7.73114556e-07]])
```


In [ ]:
logL = lambda x: -(x - 10) ** 2 - 10

def find_tangent(beta, a=0.01):
    y1 = logL(beta)
    y2 = logL(beta+a)
    x = np.array([[beta, 1], [beta+a, 1]])
    m, c = np.linalg.lstsq(x, np.array([y1, y2]))[0]
    return m, c

beta = np.linspace(2, 18)
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(beta, logL(beta), lw=2, c='black')

for beta in [7, 8.5, 9.5, 10]:
    beta_line = np.linspace(beta-2, beta+2)
    m, c = find_tangent(beta)
    y = m*beta_line + c
    ax.plot(beta_line, y, '-', c='purple', alpha=0.8)
    ax.text(beta+2.05, y[-1], r'$G({}) = {:.0f}$'.format(beta, abs(m)), fontsize=12)
    ax.vlines(beta, -24, logL(beta), linestyles='--', alpha=0.5)
    ax.hlines(logL(beta), 6, beta, linestyles='--', alpha=0.5)

ax.set(ylim=(-24, -4), xlim=(6, 13))
ax.set_xlabel('$\\beta$', fontsize=15)
ax.set_ylabel('$log \mathcal{L(\\beta)}$',
               rotation=0,
               labelpad=25,
               fontsize=15)
ax.grid(alpha=0.3)
plt.show()

In [ ]:
from statsmodels.api import Poisson

X = np.array([[1, 2, 5],
              [1, 1, 3],
              [1, 4, 2],
              [1, 5, 2],
              [1, 3, 1]])

y = np.array([1, 0, 1, 1, 0])

stats_poisson = Poisson(y, X).fit()
print(stats_poisson.summary())

```none
Optimization terminated successfully.
         Current function value: 0.675671
         Iterations 7
                          Poisson Regression Results
==============================================================================
Dep. Variable:                      y   No. Observations:                    5
Model:                        Poisson   Df Residuals:                        2
Method:                           MLE   Df Model:                            2
Date:                Wed, 26 Jul 2017   Pseudo R-squ.:                  0.2546
Time:                        15:41:38   Log-Likelihood:                -3.3784
converged:                       True   LL-Null:                       -4.5325
                                        LLR p-value:                    0.3153
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.0785      5.279     -1.151      0.250     -16.425       4.268
x1             0.9334      0.829      1.126      0.260      -0.691       2.558
x2             0.8433      0.798      1.057      0.291      -0.720       2.407
==============================================================================
```


In [ ]:
# Keep only year 2008
df = df[df['year'] == 2008]

# Add a constant
df['const'] = 1

# Variable sets
reg1 = ['const', 'lngdppc', 'lnpop', 'gattwto08']
reg2 = ['const', 'lngdppc', 'lnpop',
        'gattwto08', 'lnmcap08', 'rintr', 'topint08']
reg3 = ['const', 'lngdppc', 'lnpop', 'gattwto08', 'lnmcap08',
        'rintr', 'topint08', 'nrrents', 'roflaw']

In [ ]:
import statsmodels.api as sm

# Specify model
poisson_reg = sm.Poisson(df[['numbil0']], df[reg1],
                         missing='drop').fit(cov_type='HC0')
print(poisson_reg.summary())

```none
Warning: Maximum number of iterations has been exceeded.
         Current function value: 2.226090
         Iterations: 35
                          Poisson Regression Results
==============================================================================
Dep. Variable:                numbil0   No. Observations:                  197
Model:                        Poisson   Df Residuals:                      193
Method:                           MLE   Df Model:                            3
Date:                Wed, 26 Jul 2017   Pseudo R-squ.:                  0.8574
Time:                        15:41:38   Log-Likelihood:                -438.54
converged:                      False   LL-Null:                       -3074.7
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -29.0495      2.578    -11.268      0.000     -34.103     -23.997
lngdppc        1.0839      0.138      7.834      0.000       0.813       1.355
lnpop          1.1714      0.097     12.024      0.000       0.980       1.362
gattwto08      0.0060      0.007      0.868      0.386      -0.008       0.019
==============================================================================
```


In [ ]:
poisson_reg = sm.Poisson(df[['numbil0']], df[reg1],
                         missing='drop').fit(cov_type='HC0', maxiter=100)
print(poisson_reg.summary())

```none
Optimization terminated successfully.
         Current function value: 2.226090
         Iterations 36
                          Poisson Regression Results
==============================================================================
Dep. Variable:                numbil0   No. Observations:                  197
Model:                        Poisson   Df Residuals:                      193
Method:                           MLE   Df Model:                            3
Date:                Wed, 26 Jul 2017   Pseudo R-squ.:                  0.8574
Time:                        15:41:38   Log-Likelihood:                -438.54
converged:                       True   LL-Null:                       -3074.7
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -29.0495      2.578    -11.268      0.000     -34.103     -23.997
lngdppc        1.0839      0.138      7.834      0.000       0.813       1.355
lnpop          1.1714      0.097     12.024      0.000       0.980       1.362
gattwto08      0.0060      0.007      0.868      0.386      -0.008       0.019
==============================================================================
```


In [ ]:
from statsmodels.iolib.summary2 import summary_col

regs = [reg1, reg2, reg3]
reg_names = ['Model 1', 'Model 2', 'Model 3']
info_dict = {'Pseudo R-squared': lambda x: "{:.2f}".format(x.prsquared),
             'No. observations': lambda x: "{0:d}".format(int(x.nobs))}
regressor_order = ['const',
                   'lngdppc',
                   'lnpop',
                   'gattwto08',
                   'lnmcap08',
                   'rintr',
                   'topint08',
                   'nrrents',
                   'roflaw']
results = []

for reg in regs:
    result = sm.Poisson(df[['numbil0']], df[reg],
                        missing='drop').fit(cov_type='HC0', maxiter=100, disp=0)
    results.append(result)

results_table = summary_col(results=results,
                            float_format='%0.3f',
                            stars=True,
                            model_names=reg_names,
                            info_dict=info_dict,
                            regressor_order=regressor_order)
results_table.add_title('Table 1 - Explaining the Number of Billionaires in 2008')
print(results_table)

```none
Table 1 - Explaining the Number of Billionaires in 2008
=================================================
                  Model 1    Model 2    Model 3
-------------------------------------------------
const            -29.050*** -19.444*** -20.858***
                 (2.578)    (4.820)    (4.255)
lngdppc          1.084***   0.717***   0.737***
                 (0.138)    (0.244)    (0.233)
lnpop            1.171***   0.806***   0.929***
                 (0.097)    (0.213)    (0.195)
gattwto08        0.006      0.007      0.004
                 (0.007)    (0.006)    (0.006)
lnmcap08                    0.399**    0.286*
                            (0.172)    (0.167)
rintr                       -0.010     -0.009
                            (0.010)    (0.010)
topint08                    -0.051***  -0.058***
                            (0.011)    (0.012)
nrrents                                -0.005
                                       (0.010)
roflaw                                 0.203
                                       (0.372)
Pseudo R-squared 0.86       0.90       0.90
No. observations 197        131        131
=================================================
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
```


In [ ]:
data = ['const', 'lngdppc', 'lnpop', 'gattwto08', 'lnmcap08', 'rintr',
        'topint08', 'nrrents', 'roflaw', 'numbil0', 'country']
results_df = df[data].dropna()

# Use last model (model 3)
results_df['prediction'] = results[-1].predict()

# Calculate difference
results_df['difference'] = results_df['numbil0'] - results_df['prediction']

# Sort in descending order
results_df.sort_values('difference', ascending=False, inplace=True)

# Plot the first 15 data points
results_df[:15].plot('country', 'difference', kind='bar', figsize=(12,8), legend=False)
plt.ylabel('Number of billionaires above predicted level')
plt.xlabel('Country')
plt.show()

In [ ]:
from statsmodels.discrete.discrete_model import Probit

In [ ]:
from scipy.stats import norm

class ProbitRegression:

    def __init__(self, y, X, beta):
        self.X, self.y, self.beta = X, y, beta
        self.n, self.k = X.shape

    def mu(self):
        return norm.cdf(np.dot(self.X, self.beta.T))

    def phi(self):
        return norm.pdf(np.dot(self.X, self.beta.T))

    def logL(self):
        mu = self.mu()
        return np.sum(y * np.log(mu) + (1-y) * np.log(1-mu))

    def G(self):
        mu = self.mu()
        phi = self.phi()
        return np.sum((X.T * y*phi/mu - X.T * (1-y)*phi/(1-mu)), axis=1)

    def H(self):
        X = self.X
        beta = self.beta
        mu = self.mu()
        phi = self.phi()
        a = (phi + np.dot(X, beta.T) * mu) / mu**2
        b = (phi - np.dot(X, beta.T) * (1 - mu)) / (1-mu)**2
        return -np.dot(phi * (y*a + (1-y)*b) * X.T, X)

In [ ]:
X = np.array([[1, 2, 4],
              [1, 1, 1],
              [1, 4, 3],
              [1, 5, 6],
              [1, 3, 5]])

y = np.array([1, 0, 1, 1, 0])

# Take a guess at initial betas
beta = np.array([0.1, 0.1, 0.1])

# Create instance of Probit regression class
prob = ProbitRegression(y, X, beta)

# Run Newton-Raphson algorithm
newton_raphson(prob)

```none
Iteration_k  Log-likelihood  Theta
-----------------------------------------------------------
0            -2.37968841     ['-1.3400', '0.7750', '-0.1570']
1            -2.36875259     ['-1.5350', '0.7750', '-0.0980']
2            -2.36872942     ['-1.5460', '0.7780', '-0.0970']
3            -2.36872942     ['-1.5460', '0.7780', '-0.0970']
Number of iterations: 4
beta_hat = [-1.54625858  0.77778952 -0.09709757]
```


```none
array([-1.54625858,  0.77778952, -0.09709757])
```


In [ ]:
# Use statsmodels to verify results

print(Probit(y, X).fit().summary())

```none
Optimization terminated successfully.
         Current function value: 0.473746
         Iterations 6
                          Probit Regression Results
==============================================================================
Dep. Variable:                      y   No. Observations:                    5
Model:                         Probit   Df Residuals:                        2
Method:                           MLE   Df Model:                            2
Date:                Wed, 26 Jul 2017   Pseudo R-squ.:                  0.2961
Time:                        15:41:39   Log-Likelihood:                -2.3687
converged:                       True   LL-Null:                       -3.3651
                                        LLR p-value:                    0.3692
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5463      1.866     -0.829      0.407      -5.204       2.111
x1             0.7778      0.788      0.986      0.324      -0.768       2.323
x2            -0.0971      0.590     -0.165      0.869      -1.254       1.060
==============================================================================
```
